In [1]:
import glob
import pandas as pd
import gzip
import numpy as np
from sympy import *
from sympy import roots, solve_poly_system
import sys

In [2]:
eCAVIAR_directory = 'ukbb_height_conditioned/eCAVIAR/'
tissue = 'Muscle_Skeletal/'

LD_directory = '/net/data/GTEx/ukbb_ld/'
zfile_directory = '/net/data/GTEx/gwas/' + eCAVIAR_directory + 'eCAVIAR_zscore/' + tissue
eC_LD_directory = '/net/data/GTEx/gwas/' + eCAVIAR_directory + 'LD/' + tissue
output_directory = '/net/data/GTEx/gwas/' + eCAVIAR_directory + 'eCAVIAR_output/' + tissue

print(zfile_directory)

# zfiles = glob.glob(zfile_directory + '*.Z')
LDfiles = glob.glob(eC_LD_directory + '*.LD')
outputs = glob.glob(output_directory + '*_col')

LDfiles_split = [x.split("/")[-1][0:-3] for x in LDfiles]
LDfiles_split = set(LDfiles_split)

outputs_split = [x.split("/")[-1][3:-4] for x in outputs]
outputs_split = set(outputs_split)

missing = list(LDfiles_split - outputs_split)

len(missing)

/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/


1938

In [3]:
location = [x.split("_")[1] for x in missing]
chr_list = [x.split("_")[0] for x in missing]

data = {'chr': chr_list, 'location': location, "filenames":  missing}
zfile_df = pd.DataFrame(data)

In [4]:
#form the location_names, get the LD file names
def round_down(x):
    x = int(x)
    x -= x % 1000000
    return(x)

In [5]:
result = [round_down(x) for x in zfile_df['location']]

LD_filenames = [str(i + 1) + '_' + str(i + 3000001) for i in result]
zfile_df["LD_filename"] = LD_filenames
zfile_df['LD_filename']=zfile_df[['chr','LD_filename']].values.tolist()
zfile_df['LD_filename']=zfile_df['LD_filename'].apply('_'.join)
zfile_df['LD_filename'] = 'chr' + zfile_df['LD_filename'].astype(str)

In [6]:
old_LDfilename = ''

for index, row in zfile_df.iterrows():

    #load zfile
    zscore_file_name = zfile_directory + 'eqtl_' + row['filenames'] + '.Z'
    z_file = pd.read_csv(zscore_file_name , sep= '\t', names = ["name", "zscore"])

    print(zscore_file_name)
    
    if len(z_file) == 0:
        continue

    #add chromosome and position column to zscore dataframe
    variant_id = z_file["name"].str.split("_", n = 4, expand = True)
    z_file["chromosome"] = variant_id[0]
    z_file["position"] = variant_id[1].astype(int)
    z_file["major_allele"] = variant_id[2]
    z_file["minor_allele"] = variant_id[3]
    
    #drop duplicates
    z_file = z_file.drop_duplicates(subset=['position'])
    z_file[["name", "zscore"]].to_csv(zscore_file_name,index = False, header = None, sep='\t')
    
    print(len(z_file))

    #do the same for gwas
    gwas_zscore_file_name = zfile_directory + 'gwas_' + zscore_file_name.split("/")[-1][5:-2] + '.Z'
    gwas_z_file = pd.read_csv(gwas_zscore_file_name , sep= '\t', names = ["name", "zscore"])

    variant_id = gwas_z_file["name"].str.split("_", n = 4, expand = True)
    gwas_z_file["position"] = variant_id[1].astype(int)
    
    gwas_z_file = gwas_z_file.drop_duplicates(subset=['position'])
    gwas_z_file[["name", "zscore"]].to_csv(gwas_zscore_file_name,index = False, header = None, sep='\t')
    
    print(len(gwas_z_file))

/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_19_49122248_ENSG00000268093.1.Z
928
928
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_14_74379984_ENSG00000165898.9.Z
884
884
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_1_144897876_ENSG00000131779.6.Z
161
161
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_15_102007617_ENSG00000154227.9.Z
1116
1116
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_14_94838142_ENSG00000235706.3.Z
910
910
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_14_101198609_ENSG00000197406.6.Z
847
847
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_1_52053017_ENSG00000078618.15.Z
507
507
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_11_68415513_

/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_6_33191338_ENSG00000204261.4.Z
1093
1093
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_19_44280837_ENSG00000266921.1.Z
939
939
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_1_45502217_ENSG00000178028.9.Z
635
635
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_14_74665725_ENSG00000119630.9.Z
919
919
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_8_9104590_ENSG00000173295.3.Z
344
344
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_1_156061222_ENSG00000160753.11.Z
406
406
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_15_100660989_ENSG00000259475.1.Z
679
679
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_14_24835017_ENSG

440
440
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_9_99428825_ENSG00000224546.2.Z
537
537
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_12_123258997_ENSG00000033030.9.Z
759
759
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_1_53589801_ENSG00000157077.10.Z
729
729
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_6_109723017_ENSG00000112365.4.Z
721
721
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_11_65413855_ENSG00000254510.1.Z
513
513
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_14_74379984_ENSG00000258944.1.Z
884
884
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_11_65413855_ENSG00000172757.8.Z
513
513
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_15_48729

59
59
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_2_238400036_ENSG00000225057.2.Z
1106
1106
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_4_38606933_ENSG00000174125.3.Z
710
710
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_6_143618626_ENSG00000229017.2.Z
648
648
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_3_56666502_ENSG00000174840.8.Z
934
934
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_6_82945613_ENSG00000005700.10.Z
584
584
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_14_74665725_ENSG00000176903.3.Z
919
919
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_17_38344485_ENSG00000131771.9.Z
404
404
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_2_56350306

844
844
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_12_123258997_ENSG00000168778.7.Z
759
759
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_9_109643880_ENSG00000148143.8.Z
461
461
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_12_122828069_ENSG00000111011.13.Z
535
535
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_17_45635239_ENSG00000108465.10.Z
463
463
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_2_23896049_ENSG00000163026.7.Z
769
769
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_11_74288826_ENSG00000175581.9.Z
765
765
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_1_144897876_ENSG00000223380.3.Z
161
161
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_15_10

...

771
771
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_12_121332593_ENSG00000248008.2.Z
778
778
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_5_102706023_ENSG00000112874.5.Z
408
408
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_14_104199491_ENSG00000088808.12.Z
563
563
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_18_44773382_ENSG00000134049.3.Z
760
760
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_6_143618626_ENSG00000217648.1.Z
648
648
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_3_29665968_ENSG00000144642.16.Z
1025
1025
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_14_21538067_ENSG00000179636.10.Z
844
844
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_14

1090
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_12_123258997_ENSG00000158113.8.Z
759
759
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_12_102900041_ENSG00000111666.6.Z
603
603
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_12_122828069_ENSG00000090975.8.Z
535
535
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_14_74665725_ENSG00000119596.13.Z
919
919
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_14_101198609_ENSG00000273087.1.Z
847
847
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_5_176749358_ENSG00000160883.6.Z
495
495
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_3_128789886_ENSG00000273437.1.Z
409
409
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_1_1514

844
844
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_14_104199491_ENSG00000269963.1.Z
563
563
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_7_66318629_ENSG00000126524.5.Z
562
562
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_11_95350518_ENSG00000270689.1.Z
773
773
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_12_8831954_ENSG00000171847.6.Z
670
670
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_4_38606933_ENSG00000196355.2.Z
710
710
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_17_45635239_ENSG00000213326.4.Z
463
463
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_3_53133149_ENSG00000157445.10.Z
231
231
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_17_8053307

525
525
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_19_44280837_ENSG00000256294.3.Z
939
939
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_15_89397827_ENSG00000140543.9.Z
918
918
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_17_45635239_ENSG00000141295.9.Z
463
463
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_14_24835017_ENSG00000092098.12.Z
870
870
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_2_218604987_ENSG00000018280.12.Z
727
727
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_7_65815254_ENSG00000067601.6.Z
362
362
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_17_8024218_ENSG00000174083.13.Z
805
805
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_12_8831

559
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_17_80533079_ENSG00000176845.8.Z
961
961
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_9_17040371_ENSG00000173068.13.Z
1024
1024
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_12_50901882_ENSG00000258057.1.Z
456
456
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_1_52053017_ENSG00000134717.13.Z
507
507
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_1_156061222_ENSG00000132692.14.Z
406
406
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_17_8024218_ENSG00000175826.7.Z
805
805
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_19_16429949_ENSG00000186204.10.Z
720
720
/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_zscore/Muscle_Skeletal/eqtl_1_2433960

In [9]:
output_filename = 'height_W_eCAVIAR.sh'

line_list = []

for identifier in missing:
        
    line = '/net/home/dlee/git/caviar/CAVIAR-C++/eCAVIAR -o ' + output_directory + 'chr'
    line += identifier
    line += ' -l ' + eC_LD_directory
    line += identifier + '.LD'
    line += ' -l ' + eC_LD_directory
    line += identifier + '.LD'
    line += ' -z ' + zfile_directory+ 'gwas_'
    line += identifier + '.Z'
    line += ' -z ' + zfile_directory+ 'eqtl_'
    line += identifier + '.Z'
    
    line_list.append(line)

print(len(missing))

open(output_filename, 'w').close()

with open(output_filename, 'a') as the_file:
    the_file.write('#/bin/sh')
    the_file.write('\n')

    for i in line_list:
        the_file.write(i + '\n\n')

37
